In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
import numpy as np
import os

In [ ]:
data = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
data

In [ ]:
label_dict = {
0:"Cassava Bacterial Blight (CBB)",
1:"Cassava Brown Streak Disease (CBSD)",
2:"Cassava Green Mottle (CGM)",
3:"Cassava Mosaic Disease (CMD)",
4:"Healthy"
}
label_dict

In [ ]:
data['label'].replace(label_dict, inplace=True)

In [ ]:
data.head()

In [ ]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(data, test_size=0.3, stratify=data.label)

In [ ]:
train.head()

In [ ]:
valid.head()

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.0)
train_data = train_datagen.flow_from_dataframe(train ,directory='../input/cassava-leaf-disease-classification/train_images',
                                                    x_col="image_id" , y_col="label", target_size=(300, 300), color_mode="rgb",
                                                    class_mode="categorical",batch_size=128)
valid_data = train_datagen.flow_from_dataframe(valid ,directory='../input/cassava-leaf-disease-classification/train_images',
                                                    x_col="image_id" , y_col="label", target_size=(300, 300), color_mode="rgb",
                                                    class_mode="categorical",batch_size=32)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation = 'relu'),
    tf.keras.layers.Dense(5,activation = 'softmax')
])
model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

In [ ]:
model.fit(train_data,
          epochs = 20,
          validation_data = valid_data,
          callbacks = [callback],verbose = 1)

In [ ]:
model.save('cassava-model.h5')

In [ ]:
from keras.preprocessing import image
test_image = image.load_img('/kaggle/input/cassava-leaf-disease-classification/test_images/2216849948.jpg', 
                            target_size = (300, 300))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

In [ ]:
submission_data = pd.DataFrame(columns = ['image_id','label'])
submission_data['image_id'] = os.listdir('../input/cassava-leaf-disease-classification/test_images')
submission_data['label'] = np.argmax(result)

In [ ]:
submission_data.to_csv('submission.csv',index=False)